In [1]:
pip install efficientnet_pytorch

In [2]:
import torch, time, copy
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torchvision import datasets, models, transforms 
import os 
from PIL import Image
from efficientnet_pytorch import EfficientNet
from tqdm import tqdm
import torch.nn.functional as F

In [3]:
'''from google.colab import drive
drive.mount('/content/drive')'''

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
image_size = (300,300)
data_transforms={
                        "train":    transforms.Compose
                                            (
                                                [
                                                    transforms.Resize(image_size),
                                                    transforms.ColorJitter(hue=.05, saturation=.05),
                                                    transforms.RandomHorizontalFlip(),
                                                    transforms.RandomRotation(20, resample=Image.BILINEAR),
                                                    transforms.ToTensor(),
                                                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                                ]
                                            ),

                        "val":      transforms.Compose
                                            (
                                                [
                                                    transforms.Resize(image_size),
                                                    transforms.ToTensor(),
                                                    transforms.Normalize([0.456], [0.224])
                                                ]
                                            )
                }

 
data_dir=r"Dataset/"
 
image_datasets = {  x: datasets.ImageFolder(os.path.join(data_dir,x),data_transforms[x]) for x in ["train", "val"]  }

dataloaders             =   {}
dataloaders["train"]    =   torch.utils.data.DataLoader(image_datasets["train"], batch_size=8, shuffle=True, num_workers=2) 
dataloaders["val"]     =   torch.utils.data.DataLoader(image_datasets["val"], batch_size=8, shuffle=False, num_workers=2) 

dataset_sizes={x: len(image_datasets[x]) for x in ["train","val"]}
 
class_names=image_datasets["train"].classes
print(class_names)

num_classes=len(class_names)
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
 
val = {"loss":[],"acc":[]}
train = {"loss":[],"acc":[]}
 


['Capped', 'NoCap']


/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:1201: UserWarning: Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead
  "Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead"


In [16]:
def train_model(model, criterion, optimizer , num_epochs=10):
    start_time=time.time()
 
    best_acc= 0.0
 
    for epoch in range(num_epochs):
        print("epoch{}/{}".format(epoch,num_epochs-1))
        print("-"*10)
 
        for phase in ["train", "val"]:
            if phase =="train":
                model.train()
            else:
                model.eval()
 
            running_loss=0.0
            running_corrects=0.0
 
            for inputs,labels in tqdm(dataloaders[phase]):
                inputs=inputs.to(device)
                labels=labels.to(device)
 
                optimizer.zero_grad()
 
                with torch.set_grad_enabled(phase=="train"):
                    outputs=model(inputs)
                    _,preds=torch.max(outputs,1)
                    loss=criterion(outputs,labels)
 
                    if phase == "train":
                        loss.backward()
                        optimizer.step()
 
                running_loss+=loss.item() * inputs.size(0)
                running_corrects+=  torch.sum(preds==labels.data)
 
            epoch_loss=running_loss/dataset_sizes[phase]
            epoch_acc=running_corrects.double()/dataset_sizes[phase]
 
            if phase == "train":
              train["loss"].append(epoch_loss)
              train["acc"].append(epoch_acc.item())
            else:
              val["loss"].append(epoch_loss)
              val["acc"].append(epoch_acc.item())
 
            print("{} Loss: {:.4f} Acc: {:.4f}".format(phase,epoch_loss,epoch_acc))
 
            if phase == "val" and epoch_acc>best_acc:
                torch.save(model,"./models/efnet-b3-best.pth")
                best_acc=epoch_acc
 
    time_elapsed=time.time()- start_time
    print("training completed in {:.0f}m {:.0f}s".format(time_elapsed//60,time_elapsed%60))
    print("best val accuracy: {:4f}".format(best_acc))
 
    return model
 


In [17]:
class EffNet(nn.Module):
    def __init__(self):
        super(EffNet, self).__init__()
        self.eff_net = EfficientNet.from_pretrained('efficientnet-b3', num_classes=2)
        self.eff_net.set_swish(memory_efficient=False)
    def forward(self, x):
        x = self.eff_net(x)
        x = F.softmax(x, dim=1)
        return x

In [18]:

model = EffNet()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=1e-3)

model_ft = train_model(model, criterion, optimizer, num_epochs=15)
 
torch.save(model_ft,"./models/efnet-b3-last.pth")

  0%|          | 0/117 [00:00<?, ?it/s]

Loaded pretrained weights for efficientnet-b3
epoch0/14
----------


  0%|          | 0/30 [00:00<?, ?it/s]

train Loss: 0.3803 Acc: 0.9354


100%|██████████| 30/30 [00:03<00:00,  9.90it/s]


...

val Loss: 0.3532 Acc: 0.9573
epoch14/14
----------


  0%|          | 0/30 [00:00<?, ?it/s]

train Loss: 0.3592 Acc: 0.9537


100%|██████████| 30/30 [00:03<00:00,  9.88it/s]


val Loss: 0.3525 Acc: 0.9615
training completed in 7m 9s
best val accuracy: 1.000000
